<a href="https://colab.research.google.com/github/pxs1990/python_data_science/blob/main/starSchema_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3

Reading CSV files in pandas dataframe

In [2]:
# Read dimension tables from CSV files
creditinfo_dim_df = pd.read_csv('/content/Credit_Info_Dim.csv')
demographic_dim_df = pd.read_csv('/content/Demographic_Dimension.csv')
loanapp_dim_df = pd.read_csv('/content/Loan_Application_Dimension.csv')
otherdetail_dim_df = pd.read_csv('/content/Other_Details_Dim.csv')

# Read fact table from CSV file
loan_fact_df = pd.read_csv('/content/Loan_Fact_Table.csv')


In [3]:
# Connect to SQLite database
conn = sqlite3.connect('risk_star_schema.db')

In [4]:
# Define schema for dimension tables
credit_info_schema = """
CREATE TABLE Credit_Info_Dim (
    Credit_Info_ID TEXT PRIMARY KEY,
    ID INTEGER,
    Credit_Worthiness TEXT,
    Open_Credit TEXT,
    Credit_Type TEXT,
    Co_Applicant_Credit_Type TEXT
);
"""

demographics_schema = """
CREATE TABLE Demographics_Dim (
    Demographics_ID TEXT PRIMARY KEY,
    ID INTEGER,
    Year INTEGER,
    Gender TEXT,
    Age TEXT
);
"""

loan_details_schema = """
CREATE TABLE Loan_Details_Dim (
    Loan_Details_ID TEXT PRIMARY KEY,
    ID INTEGER,
    Loan_Limit TEXT,
    Approv_in_adv TEXT,
    Loan_Type TEXT,
    Loan_Purpose TEXT,
    Occupancy_Type TEXT,
    Total_Units TEXT
);
"""



other_details_schema = """
CREATE TABLE Other_Details_Dim (
    Other_Details_ID TEXT PRIMARY KEY,
    ID INTEGER,
    Business_or_Commercial TEXT,
    Submission_of_Application TEXT,
    Region TEXT,
    Security_Type TEXT
);
"""

# Define schema for fact table
loan_fact_schema = """
CREATE TABLE Loan_Fact (
    ID INTEGER PRIMARY KEY,
    Demographics_ID TEXT,
    Loan_Details_ID TEXT,
    Credit_Info_ID TEXT,
    Other_Details_ID TEXT,
    Income FLOAT,
    LTV FLOAT,
    DTIR1 FLOAT,
    Loan_Amount FLOAT,
    Rate_of_Interest FLOAT,
    Term INTEGER,
    Property_Value FLOAT,
    Credit_Score INTEGER,
    Status INTEGER,
    FOREIGN KEY (Demographics_ID) REFERENCES Demographics_Dim(Demographics_ID),
    FOREIGN KEY (Loan_Details_ID) REFERENCES Loan_Details_Dim(Loan_Details_ID),
    FOREIGN KEY (Credit_Info_ID) REFERENCES Credit_Info_Dim(Credit_Info_ID),
    FOREIGN KEY (Other_Details_ID) REFERENCES Other_Details_Dim(Other_Details_ID)
);
"""

In [5]:
# Execute schema creation queries
conn.executescript(demographics_schema)
conn.executescript(loan_details_schema)
conn.executescript(credit_info_schema)
conn.executescript(other_details_schema)
conn.executescript(loan_fact_schema)

In [6]:
# Save dimension tables to SQLite
demographic_dim_df.to_sql('Demographics_Dim', conn, if_exists='replace', index=False)
loanapp_dim_df.to_sql('Loan_Details_Dim', conn, if_exists='replace', index=False)
creditinfo_dim_df.to_sql('Credit_Info_Dim', conn, if_exists='replace', index=False)
otherdetail_dim_df.to_sql('Other_Details_Dim', conn, if_exists='replace', index=False)

# Save fact table to SQLite
loan_fact_df.to_sql('Loan_Fact', conn, if_exists='replace', index=False)

148670

# **Querrying a table:**

In [7]:
# Create a cursor object
cur = conn.cursor()

# Execute a query to select data from the Loan_Fact table
cur.execute("SELECT * FROM Credit_Info_Dim LIMIT 20")

# Fetch the results
result = cur.fetchall()

# Print the results
for row in result:
    print(row)

('C1', 24890, 'l1', 'nopc', 'EXP', 'CIB')
('C2', 24891, 'l1', 'nopc', 'EQUI', 'EXP')
('C3', 24892, 'l1', 'nopc', 'EXP', 'CIB')
('C4', 24893, 'l1', 'nopc', 'EXP', 'CIB')
('C5', 24894, 'l1', 'nopc', 'CRIF', 'EXP')
('C6', 24895, 'l1', 'nopc', 'EXP', 'EXP')
('C7', 24896, 'l1', 'nopc', 'EXP', 'EXP')
('C8', 24897, 'l1', 'nopc', 'CIB', 'CIB')
('C9', 24898, 'l1', 'nopc', 'CIB', 'EXP')
('C10', 24899, 'l1', 'nopc', 'CIB', 'EXP')
('C11', 24900, 'l2', 'nopc', 'EXP', 'CIB')
('C12', 24901, 'l1', 'nopc', 'EXP', 'EXP')
('C13', 24902, 'l1', 'nopc', 'CRIF', 'EXP')
('C14', 24903, 'l1', 'nopc', 'CRIF', 'EXP')
('C15', 24904, 'l1', 'nopc', 'CIB', 'CIB')
('C16', 24905, 'l1', 'nopc', 'EXP', 'CIB')
('C17', 24906, 'l1', 'nopc', 'CRIF', 'EXP')
('C18', 24907, 'l1', 'nopc', 'CRIF', 'CIB')
('C19', 24908, 'l1', 'nopc', 'CIB', 'CIB')
('C20', 24909, 'l1', 'nopc', 'CIB', 'EXP')


In [8]:
# Replace 'your_table_name' with the name of your table
table_name = 'Credit_Info_Dim'

# Execute the PRAGMA statement to fetch column information
cur.execute(f"PRAGMA table_info({table_name})")

# Fetch all rows (columns) from the cursor
rows = cur.fetchall()

# Extract and print column names
column_names = [row[1] for row in rows]
print(column_names)

['Credit_Info_ID', 'ID', 'Credit_Worthiness', 'open_credit', 'credit_type', 'co_applicant_credit_type']


In [12]:
# Execute the SQL query to join tables
cur.execute("""
    CREATE TABLE IF NOT EXISTS myTable AS
    SELECT
        f.ID,
        d.Year,
        d.Gender,
        d.Age,
        f.Income,
        f.LTV,
        f.DTIR1,
        ld.Loan_Limit,
        ld.Approv_in_adv,
        ld.Loan_Type,
        ld.Loan_Purpose,
        ld.Occupancy_Type,
        ld.Total_Units,
        ci.Credit_Worthiness,
        ci.Open_Credit,
        ci.Credit_Type,
        ci.Co_Applicant_Credit_Type,
        od.Business_or_Commercial,
        od.Submission_of_Application,
        od.Region,
        od.Security_Type,
        f.Status
    FROM
        Loan_Fact f
    JOIN Demographics_Dim d ON f.ID = d.ID
    JOIN Loan_Details_Dim ld ON f.ID = ld.ID
    JOIN Credit_Info_Dim ci ON f.ID = ci.ID
    JOIN Other_Details_Dim od ON f.ID = od.ID
""")

In [14]:
cur.execute("Select * from myTable limit 20")
# Fetch and print the results
rows = cur.fetchall()
for row in rows:
    print(row)

(24890, 2019, 'Sex Not Available', '25-34', 1740.0, 98.72881356, 45.0, 'cf', 'nopre', 'type1', 'p1', 'pr', '1U', 'l1', 'nopc', 'EXP', 'CIB', 'nob/c', 'to_inst', 'south', 'direct', 1)
(24891, 2019, 'Male', '55-64', 4980.0, None, None, 'cf', 'nopre', 'type2', 'p1', 'pr', '1U', 'l1', 'nopc', 'EQUI', 'EXP', 'b/c', 'to_inst', 'North', 'direct', 1)
(24892, 2019, 'Male', '35-44', 9480.0, 80.01968504, 46.0, 'cf', 'pre', 'type1', 'p1', 'pr', '1U', 'l1', 'nopc', 'EXP', 'CIB', 'nob/c', 'to_inst', 'south', 'direct', 0)
(24893, 2019, 'Male', '45-54', 11880.0, 69.3768997, 42.0, 'cf', 'nopre', 'type1', 'p4', 'pr', '1U', 'l1', 'nopc', 'EXP', 'CIB', 'nob/c', 'not_inst', 'North', 'direct', 0)
(24894, 2019, 'Joint', '25-34', 10440.0, 91.88654354, 39.0, 'cf', 'pre', 'type1', 'p1', 'pr', '1U', 'l1', 'nopc', 'CRIF', 'EXP', 'nob/c', 'not_inst', 'North', 'direct', 0)
(24895, 2019, 'Joint', '35-44', 10080.0, 70.08928571, 40.0, 'cf', 'pre', 'type1', 'p1', 'pr', '1U', 'l1', 'nopc', 'EXP', 'EXP', 'nob/c', 'not_in

In [15]:
# Close the cursor and connection
cur.close()
conn.close()